<a href="https://colab.research.google.com/github/gustavor10silva/alura_imersao_dados_4/blob/main/4_modelo_de_regressao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2 Preparando os dados

Primeiro, vamos selecionar apenas as variáveis numéricas para utilizarmos no treinamento do nosso modelo de regressão:

In [ ]:
df_imoveis_ibge = pd.read_csv('/content/drive/MyDrive/Alura/imersao_dados4/df_imoveis_ibge.csv')
df_imoveis_ibge.drop(columns = ['Unnamed: 0'], inplace = True)
df_imoveis_ibge.head()

,endereco,Rua,Bairro,Cidade,Metragem,Quartos,Banheiros,Vagas,Valor,endereco_geolocalizado,...,domicilio01_sp1_V002,V012,domicilio01_sp1_V017,domicilio01_sp1_V023,domicilio01_sp1_V035,domicilio01_sp1_V043,pessoa01_sp1_Cod_setor,pessoa01_sp1_V001,pessoa03_sp1_Cod_setor,pessoa03_sp1_V001
0,"Rua Caramuru, 100 - Saúde, São Paulo","Rua Caramuru, 100",Saúde,São Paulo,380,9,4,4,3700000,"100, Rua Caramuru, Chácara Inglesa, Saúde, São...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Alameda Franca, 587 - Jardim Paulista, São Paulo","Alameda Franca, 587",Jardim Paulista,São Paulo,137,9,2,1,2600000,"Alameda Franca, Cerqueira César, Jardim Paulis...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Alameda Franca, 587 - Jardim Paulista, São Paulo","Alameda Franca, 587",Jardim Paulista,São Paulo,142,9,2,1,2600000,"Alameda Franca, Cerqueira César, Jardim Paulis...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Alameda Franca - Jardim Paulista, São Paulo",Alameda Franca,Jardim Paulista,São Paulo,150,9,2,1,2600000,"Alameda Franca, Cerqueira César, Jardim Paulis...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Alameda Franca - Jardim Paulista, São Paulo",Alameda Franca,Jardim Paulista,São Paulo,155,9,2,1,2600000,"Alameda Franca, Cerqueira César, Jardim Paulis...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_imoveis_ibge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8975 entries, 0 to 8974
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   endereco                   5998 non-null   object 
 1   Rua                        6008 non-null   object 
 2   Bairro                     8965 non-null   object 
 3   Cidade                     8975 non-null   object 
 4   Metragem                   8975 non-null   int64  
 5   Quartos                    8975 non-null   int64  
 6   Banheiros                  8975 non-null   int64  
 7   Vagas                      8975 non-null   int64  
 8   Valor                      8975 non-null   int64  
 9   endereco_geolocalizado     5898 non-null   object 
 10  endereco_point             5898 non-null   object 
 11  latitude                   5898 non-null   float64
 12  longitude                  5898 non-null   float64
 13  point                      5898 non-null   objec

In [ ]:
df_num = df_imoveis_ibge.drop(columns = ['endereco',
                                                  'Rua',
                                                  'Bairro',
                                                  'Cidade',
                                                  'setor_censo',
                                                  'basico_sp1_Cod_setor',
                                                  'endereco_geolocalizado',
                                                  'endereco_point',
                                                  'latitude',
                                                  'longitude',
                                                  'point',
                                                  'pessoa01_sp1_Cod_setor',
                                                  'pessoa03_sp1_Cod_setor',
                                                  'domicilio01_sp1_Cod_setor'])

In [ ]:
df_num

,Metragem,Quartos,Banheiros,Vagas,Valor,basico_sp1_Situacao_setor,basico_sp1_V001,basico_sp1_V002,basico_sp1_V003,basico_sp1_V004,...,basico_sp1_V006,domicilio01_sp1_V001,domicilio01_sp1_V002,V012,domicilio01_sp1_V017,domicilio01_sp1_V023,domicilio01_sp1_V035,domicilio01_sp1_V043,pessoa01_sp1_V001,pessoa03_sp1_V001
0,380,9,4,4,3700000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,137,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,142,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,150,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8970,107,1,1,1,188682,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8971,112,1,1,1,188682,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8972,45,1,1,1,160000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8973,61,1,1,1,147288,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_num[df_num['basico_sp1_V001'].notnull()]

,Metragem,Quartos,Banheiros,Vagas,Valor,basico_sp1_Situacao_setor,basico_sp1_V001,basico_sp1_V002,basico_sp1_V003,basico_sp1_V004,...,basico_sp1_V006,domicilio01_sp1_V001,domicilio01_sp1_V002,V012,domicilio01_sp1_V017,domicilio01_sp1_V023,domicilio01_sp1_V035,domicilio01_sp1_V043,pessoa01_sp1_V001,pessoa03_sp1_V001
28,727,7,6,7,5799000,1.0,85.0,316.0,3.72,2.35,...,2.034546e+08,85.0,85.0,85,82,0,85,85,295.0,316.0
29,732,7,6,7,5799000,1.0,85.0,316.0,3.72,2.35,...,2.034546e+08,85.0,85.0,85,82,0,85,85,295.0,316.0
35,800,7,7,4,3500000,1.0,48.0,164.0,3.42,2.76,...,1.671157e+08,48.0,48.0,48,48,0,48,48,148.0,164.0
54,450,7,8,6,2199997,1.0,182.0,581.0,3.19,2.61,...,1.714286e+07,182.0,182.0,181,182,0,182,182,556.0,581.0
55,455,7,8,6,2199997,1.0,182.0,581.0,3.19,2.61,...,1.714286e+07,182.0,182.0,181,182,0,182,182,556.0,581.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8949,95,1,1,1,350000,1.0,177.0,621.0,3.51,3.36,...,2.203520e+06,177.0,177.0,177,177,0,173,177,555.0,621.0
8956,29,1,1,1,300000,1.0,289.0,929.0,3.21,2.89,...,1.387690e+06,289.0,289.0,289,283,0,289,289,854.0,929.0
8957,50,1,1,1,289000,1.0,207.0,639.0,3.09,2.21,...,1.856815e+06,207.0,207.0,206,207,0,207,207,569.0,639.0
8963,100,1,1,1,250000,1.0,154.0,599.0,3.89,4.57,...,2.493575e+06,154.0,154.0,154,142,0,154,154,508.0,599.0


Algumas colunas possuem a string 'X', então vamos excluir as linhas onde acontece isso:

In [ ]:
df_num.loc[(df_num['V012'] == 'X')
                    | (df_num['domicilio01_sp1_V017'] == 'X')
                    | (df_num['domicilio01_sp1_V023'] == 'X')
                    | (df_num['domicilio01_sp1_V035'] == 'X')
                    | (df_num['domicilio01_sp1_V043'] == 'X')]

,Metragem,Quartos,Banheiros,Vagas,Valor,basico_sp1_Situacao_setor,basico_sp1_V001,basico_sp1_V002,basico_sp1_V003,basico_sp1_V004,...,basico_sp1_V006,domicilio01_sp1_V001,domicilio01_sp1_V002,V012,domicilio01_sp1_V017,domicilio01_sp1_V023,domicilio01_sp1_V035,domicilio01_sp1_V043,pessoa01_sp1_V001,pessoa03_sp1_V001
1794,503,4,3,4,4350000,1.0,6.0,29.0,4.83,3.77,...,6.505242e+09,6.0,6.0,X,X,X,X,X,27.0,29.0


In [ ]:
df_num.drop([1794], inplace=True) # excluindo a linha 1784
df_num.reset_index(inplace = True) # resetando o index
df_num.drop(columns = ['index'], inplace = True)
df_num # o dataframe tinha 8975 linhas e agora tem 8974

,Metragem,Quartos,Banheiros,Vagas,Valor,basico_sp1_Situacao_setor,basico_sp1_V001,basico_sp1_V002,basico_sp1_V003,basico_sp1_V004,...,basico_sp1_V006,domicilio01_sp1_V001,domicilio01_sp1_V002,V012,domicilio01_sp1_V017,domicilio01_sp1_V023,domicilio01_sp1_V035,domicilio01_sp1_V043,pessoa01_sp1_V001,pessoa03_sp1_V001
0,380,9,4,4,3700000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,137,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,142,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,150,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155,9,2,1,2600000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8969,107,1,1,1,188682,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8970,112,1,1,1,188682,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8971,45,1,1,1,160000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8972,61,1,1,1,147288,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8974 entries, 0 to 8973
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Metragem                   8974 non-null   int64  
 1   Quartos                    8974 non-null   int64  
 2   Banheiros                  8974 non-null   int64  
 3   Vagas                      8974 non-null   int64  
 4   Valor                      8974 non-null   int64  
 5   basico_sp1_Situacao_setor  1359 non-null   float64
 6   basico_sp1_V001            1359 non-null   float64
 7   basico_sp1_V002            1359 non-null   float64
 8   basico_sp1_V003            1359 non-null   float64
 9   basico_sp1_V004            1359 non-null   float64
 10  basico_sp1_V005            1359 non-null   float64
 11  basico_sp1_V006            1359 non-null   float64
 12  domicilio01_sp1_V001       1359 non-null   float64
 13  domicilio01_sp1_V002       1359 non-null   float

Note que as colunas

* V012
* domicilio01_sp1_V017
* domicilio01_sp1_V023
* domicilio01_sp1_V035
* domicilio01_sp1_V043

São do tipo object. Então elas possuem dados não numéricos. Vamos resolver isso:

In [ ]:
df_num = df_num.astype('float64')
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8974 entries, 0 to 8973
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Metragem                   8974 non-null   float64
 1   Quartos                    8974 non-null   float64
 2   Banheiros                  8974 non-null   float64
 3   Vagas                      8974 non-null   float64
 4   Valor                      8974 non-null   float64
 5   basico_sp1_Situacao_setor  1359 non-null   float64
 6   basico_sp1_V001            1359 non-null   float64
 7   basico_sp1_V002            1359 non-null   float64
 8   basico_sp1_V003            1359 non-null   float64
 9   basico_sp1_V004            1359 non-null   float64
 10  basico_sp1_V005            1359 non-null   float64
 11  basico_sp1_V006            1359 non-null   float64
 12  domicilio01_sp1_V001       1359 non-null   float64
 13  domicilio01_sp1_V002       1359 non-null   float

In [ ]:
df_num.to_csv('/content/drive/MyDrive/Alura/imersao_dados4/df_num.csv')

In [ ]:
dados_teste = df_num.dropna()
dados_teste

,Metragem,Quartos,Banheiros,Vagas,Valor,basico_sp1_Situacao_setor,basico_sp1_V001,basico_sp1_V002,basico_sp1_V003,basico_sp1_V004,...,basico_sp1_V006,domicilio01_sp1_V001,domicilio01_sp1_V002,V012,domicilio01_sp1_V017,domicilio01_sp1_V023,domicilio01_sp1_V035,domicilio01_sp1_V043,pessoa01_sp1_V001,pessoa03_sp1_V001
28,727.0,7.0,6.0,7.0,5799000.0,1.0,85.0,316.0,3.72,2.35,...,2.034546e+08,85.0,85.0,85.0,82.0,0.0,85.0,85.0,295.0,316.0
29,732.0,7.0,6.0,7.0,5799000.0,1.0,85.0,316.0,3.72,2.35,...,2.034546e+08,85.0,85.0,85.0,82.0,0.0,85.0,85.0,295.0,316.0
35,800.0,7.0,7.0,4.0,3500000.0,1.0,48.0,164.0,3.42,2.76,...,1.671157e+08,48.0,48.0,48.0,48.0,0.0,48.0,48.0,148.0,164.0
54,450.0,7.0,8.0,6.0,2199997.0,1.0,182.0,581.0,3.19,2.61,...,1.714286e+07,182.0,182.0,181.0,182.0,0.0,182.0,182.0,556.0,581.0
55,455.0,7.0,8.0,6.0,2199997.0,1.0,182.0,581.0,3.19,2.61,...,1.714286e+07,182.0,182.0,181.0,182.0,0.0,182.0,182.0,556.0,581.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8948,95.0,1.0,1.0,1.0,350000.0,1.0,177.0,621.0,3.51,3.36,...,2.203520e+06,177.0,177.0,177.0,177.0,0.0,173.0,177.0,555.0,621.0
8955,29.0,1.0,1.0,1.0,300000.0,1.0,289.0,929.0,3.21,2.89,...,1.387690e+06,289.0,289.0,289.0,283.0,0.0,289.0,289.0,854.0,929.0
8956,50.0,1.0,1.0,1.0,289000.0,1.0,207.0,639.0,3.09,2.21,...,1.856815e+06,207.0,207.0,206.0,207.0,0.0,207.0,207.0,569.0,639.0
8962,100.0,1.0,1.0,1.0,250000.0,1.0,154.0,599.0,3.89,4.57,...,2.493575e+06,154.0,154.0,154.0,142.0,0.0,154.0,154.0,508.0,599.0


# 3 Separando dados de treino e teste

In [ ]:
X = dados_teste.drop(columns = ['Valor'])
y = dados_teste['Valor']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled, X_test_scaled

(array([[-0.00196903, -0.42269629, -0.98155631, ..., -1.18340666,
         -1.18880667, -0.75879869],
        [-0.57535141,  0.60578934, -0.40971493, ..., -1.39566836,
         -1.42691938, -1.48650055],
        [-1.00538819, -0.42269629, -0.40971493, ...,  0.66905904,
         -0.21594617, -0.23358778],
        ...,
        [-0.88593353, -1.45118193, -0.98155631, ...,  0.90061726,
          1.47805568,  1.45594611],
        [-0.2599911 , -0.42269629,  0.16212646, ..., -0.08350515,
          0.28749213,  0.27896745],
        [-0.71391882, -0.42269629, -0.40971493, ..., -0.1413947 ,
         -0.13430753, -0.12601446]]),
 array([[ 0.97755919,  1.63427498,  3.02133338, ..., -0.85536586,
         -0.61733617, -0.56263558],
        [-0.33644209, -0.42269629, -0.40971493, ..., -0.52732506,
         -0.44725566, -0.36647247],
        [-1.24429752, -1.45118193, -0.98155631, ..., -0.21858077,
         -0.70577803, -0.73981516],
        ...,
        [ 1.19257759,  0.60578934,  1.30580923, ..., -

# 4 Modelos de regressão

## 4.1 Regressão Linear

In [ ]:
reg = LinearRegression().fit(X_train_scaled, y_train)
y_train_pred  = reg.predict(X_train_scaled)
y_train_pred

array([ 1.79813036e+06,  2.16950128e+06,  5.70559426e+05,  8.80730353e+05,
        1.58878555e+06, -2.88294706e+05,  4.38903337e+05,  2.88324337e+06,
        2.12305895e+06,  6.31741929e+05,  7.00733397e+05,  8.54472083e+05,
       -4.15176510e+05,  2.61509496e+06,  5.74673608e+06,  2.96403351e+06,
        3.33472189e+06,  2.95786280e+06,  4.63592829e+06,  1.95131645e+06,
        1.83700554e+06,  3.76086306e+06,  1.62292396e+06,  1.69289273e+06,
        4.54446106e+06,  5.74627135e+06,  3.34938260e+06,  9.48933940e+05,
        5.32300605e+06,  8.09046461e+06,  4.15753996e+06,  4.79317683e+05,
        7.52422921e+06,  1.49322825e+06,  2.39245375e+06,  2.83380965e+05,
        3.03984749e+05,  1.13017306e+06,  3.33898652e+06,  3.67860842e+06,
        1.19618104e+06,  4.50677457e+06,  8.52195130e+05,  6.09054721e+05,
        8.48821116e+05,  8.43518939e+05,  1.86549963e+06,  1.59487387e+06,
        3.43930837e+06,  1.49953969e+06,  3.02654869e+06,  2.21104985e+06,
        2.17782334e+06,  

In [ ]:
y_test_pred = reg.predict(X_test_scaled)
y_test_pred

array([ 4.24625079e+06,  1.85104421e+06,  1.03573039e+06,  4.03466918e+06,
        1.80189748e+06,  2.84207280e+06,  2.11250998e+06,  2.66480315e+06,
        3.63134571e+05,  6.07951825e+06,  2.98933326e+06,  7.75160305e+05,
        1.90098833e+06,  6.29078370e+06,  2.73261368e+06,  4.40020009e+06,
        3.75602025e+06,  1.21562237e+06,  3.87044197e+06,  3.26920975e+06,
        4.07798722e+06,  7.41517546e+05,  3.07579750e+06,  4.17550091e+05,
        4.05832515e+06,  1.57228049e+06,  1.48995473e+06,  3.69451695e+06,
        2.24273422e+06,  9.89077653e+05,  1.63447795e+06,  1.11740989e+06,
        4.54783208e+06,  2.40607728e+06,  7.57185776e+06,  4.11550945e+05,
        7.25600261e+05,  9.73137091e+05,  1.29581456e+06,  3.21464589e+06,
        7.05311838e+05,  1.80977223e+06,  1.18922807e+06,  1.61968357e+06,
        3.52265398e+06,  5.74711707e+05,  2.13567729e+06,  9.35637980e+06,
        5.47885628e+06,  3.26227444e+06,  1.49392190e+06,  2.94331175e+06,
        2.88425795e+06,  

In [ ]:
df_results_reg_lin = pd.DataFrame(list(zip(y_test_pred, y_test)), columns = ['y_test_pred','y_test'])
df_results_reg_lin

,y_test_pred,y_test
0,4.246251e+06,4800000.0
1,1.851044e+06,636000.0
2,1.035730e+06,395000.0
3,4.034669e+06,1600000.0
4,1.801897e+06,721000.0
...,...,...
444,2.140686e+06,1970000.0
445,2.962640e+06,2450000.0
446,4.986250e+06,3200000.0
447,3.469576e+06,5300000.0


In [ ]:
df_results_reg_lin = df_results_reg_lin.astype(int)
df_results_reg_lin

,y_test_pred,y_test
0,4246250,4800000
1,1851044,636000
2,1035730,395000
3,4034669,1600000
4,1801897,721000
...,...,...
444,2140686,1970000
445,2962640,2450000
446,4986250,3200000
447,3469576,5300000


In [ ]:
df_results_reg_lin.to_csv('/content/drive/MyDrive/Alura/imersao_dados4/df_results_reg_lin.csv')

# 2.2 Random Forest

In [ ]:
regr = RandomForestRegressor(n_estimators = 500,
                             #max_depth = 2,
                             random_state=0)
regr.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=500, random_state=0)

In [ ]:
y_test_pred_rf = regr.predict(X_test_scaled)

df_results_random_forest = pd.DataFrame(list(zip(y_test_pred_rf, y_test)), columns = ['y_test_pred_rf','y_test'])
df_results_random_forest = df_results_random_forest.astype(int)
df_results_random_forest

,y_test_pred_rf,y_test
0,4847700,4800000
1,1066715,636000
2,460688,395000
3,2100019,1600000
4,1604422,721000
...,...,...
444,1733182,1970000
445,4365253,2450000
446,4314212,3200000
447,3128392,5300000


In [ ]:
df_results_random_forest.to_csv('/content/drive/MyDrive/Alura/imersao_dados4/df_results_random_forest.csv')

# 5 Anotações

* Tentar prever as colunas com nan (video discord)
* Usar um xgboost
* Tentar usar os 8k dados sem os dados do censo
* SVM ou redes neurais
* Random forests seleção de variáveis
* IDH e preço m2 por bairro